In [1]:
import os
import pandas as pd
from tqdm import tqdm
import gzip
from os import listdir
from os.path import isfile, join
from chardet import detect
import re
import numpy as np
import shutil

In [2]:
### txt 폴더 안 폴더 개수 
spk_files = listdir('./VCTK-Corpus-0.92/txt')
print(len(spk_files))
spk_files[:5]

109


['p225', 'p226', 'p227', 'p228', 'p229']

In [52]:
### wav 폴더 안 폴더 개수 
### wav 폴더 안 폴더 개수가 txt 보다 하나 더 많아서 한 개 제거
wav_files = listdir('./VCTK-Corpus-0.92/wav48_silence_trimmed')
print(len(wav_files))
wav_files[:5]

109


['p225', 'p226', 'p227', 'p228', 'p229']

In [53]:
data_path = './VCTK-Corpus-0.92/wav48_silence_trimmed/'

In [54]:
# wav 파일 mic1, mic2 분류
for i in wav_files:
    wav_path = data_path + i
    file_li = listdir(wav_path)
    for j in file_li:
        split_name = j.split('.')[0]
        num = split_name[-1]
        if num == '1':
            des_path_1 = wav_path + '/mic1'
            if not os.path.exists(des_path_1):
                os.makedirs(des_path_1)
            os.rename(wav_path +'/' +j, des_path_1 +'/' +j)
        else:
            des_path_2 = wav_path + '/mic2'
            if not os.path.exists(des_path_2):
                os.makedirs(des_path_2)
            os.rename(wav_path +'/' +j, des_path_2 +'/' +j)

In [61]:
import shutil

In [62]:
data_path = './VCTK-Corpus-0.92/txt/'

In [63]:
for i in tqdm(spk_files, mininterval=1):
    txt_path = data_path + i
    txt_li = listdir(txt_path)
    for j in txt_li:
        shutil.copy(txt_path+'/'+j, data_path+'/all_txt/'+j)

100%|████████████████████████████████████████████████████████████████████████████████| 109/109 [03:45<00:00,  2.07s/it]


In [67]:
data_path = './VCTK-Corpus-0.92/wav48_silence_trimmed/'

In [69]:
for i in tqdm(spk_files, mininterval=1):
    wav_path = data_path + i + '/mic1'
    wav_li = listdir(wav_path)
    for j in wav_li:
        shutil.copy(wav_path+'/'+j, data_path+'/all_wav/'+j)

100%|████████████████████████████████████████████████████████████████████████████████| 109/109 [06:20<00:00,  3.49s/it]


In [70]:
txt_files = listdir('./VCTK-Corpus-0.92/txt/all_txt')
print(len(txt_files))
txt_files[:5]

44583


['p225_001.txt',
 'p225_002.txt',
 'p225_003.txt',
 'p225_004.txt',
 'p225_005.txt']

In [71]:
wav_files = listdir('./VCTK-Corpus-0.92/wav48_silence_trimmed/all_wav')
print(len(wav_files))
wav_files[:5]

44283


['p225_001_mic1.flac',
 'p225_002_mic1.flac',
 'p225_003_mic1.flac',
 'p225_004_mic1.flac',
 'p225_005_mic1.flac']

In [72]:
txt_files_df =  pd.DataFrame(txt_files)
wav_files_df =  pd.DataFrame(wav_files)

In [73]:
txt_files_df.to_csv("./텍스트.csv", encoding='utf-8-sig', header=None, index=False)
wav_files_df.to_csv("./오디오.csv", encoding='utf-8-sig', header=None, index=False)

In [74]:
all_txt_files = listdir('./VCTK-Corpus-0.92/txt/all_txt')
print(len(all_txt_files))
all_txt_files[:5]

44283


['p225_001.txt',
 'p225_002.txt',
 'p225_003.txt',
 'p225_004.txt',
 'p225_005.txt']

In [77]:
all_txt = []

for i in tqdm(all_txt_files, mininterval=1):
    f = open('./VCTK-Corpus-0.92/txt/all_txt/' + i, 'r')
    while True:
        line = f.readline()
        if not line: break
        all_txt.append(line.strip('\n'))
    f.close()

100%|███████████████████████████████████████████████████████████████████████████| 44283/44283 [02:36<00:00, 282.78it/s]


In [78]:
print(len(all_txt))
all_txt[:5]

44283


['Please call Stella.',
 'Ask her to bring these things with her from the store.',
 'Six spoons of fresh snow peas, five thick slabs of blue cheese, and maybe a snack for her brother Bob.',
 'We also need a small plastic snake and a big toy frog for the kids.',
 'She can scoop these things into three red bags, and we will go meet her Wednesday at the train station.']

In [79]:
all_txt_df = pd.DataFrame(all_txt)

In [80]:
all_txt_df.to_csv("./vctk_text.csv", encoding='utf-8-sig', header=None, index=False)

In [1]:
txt_path = './vctk_text.xlsx'

In [2]:
import openpyxl as xl
import json
from collections import OrderedDict

In [4]:
wb = xl.load_workbook(txt_path)
ws = wb.active

In [5]:
files = listdir('./VCTK-Corpus-0.92/wav48_silence_trimmed/all_wav')
print(len(files))
files[0]

44283


'p225_001_mic1.flac'

In [6]:
def clean_text(text):
    pattern = '(\.flac)$'
    text = re.sub(pattern=pattern, repl='', string=text)
    text = text.strip()
    text = " ".join(text.split())
    return text  

In [7]:
wav_files = []

for i in files:
    j = "English/VCTK/item/" + i
    k = clean_text(j)
    wav_files.append(k)

In [8]:
wav_files[0]

'English/VCTK/item/p225_001_mic1'

In [9]:
json_data = OrderedDict()

In [10]:
for r in ws.rows:
        if str(r[0].value).isalpha():
            continue
        if r[0].value is not None:
            json_data[wav_files[r[0].value-1]] = r[2].value

In [12]:
with open('vctk_alignment.json', 'w', encoding="utf-8") as make_file:
        json.dump(json_data, make_file, ensure_ascii=False, indent="\t")

In [2]:
### 화자별로 jason 따로 만들기

In [11]:
### 화자별로 wav 나누기

In [33]:
wav_files = listdir('./VCTK-Corpus-0.92/화자별/wav/all_wav')
print(len(wav_files))
wav_files[:5]

FileNotFoundError: [WinError 3] 지정된 경로를 찾을 수 없습니다: './VCTK-Corpus-0.92/화자별/wav/all_wav'

In [4]:
data_path = './VCTK-Corpus-0.92/화자별/wav/'

In [6]:
speaker_li = []

for i in wav_files:
    split_name = i.split('.')[0]
    speaker = split_name[:4]
    speaker_li.append(speaker)

In [8]:
my_set = set(speaker_li)
speaker_li_edit = list(my_set)
print(len(speaker_li_edit))

108


In [10]:
for i in wav_files:
    split_name = i.split('.')[0]
    speaker = split_name[:4]
    for j in speaker_li_edit:
        if speaker == j:
            des_path_1 = data_path + '/' + j
            if not os.path.exists(des_path_1):
                os.makedirs(des_path_1)
            os.rename(data_path +'all_wav/'+i, des_path_1+'/'+i)

In [12]:
### 화자별로 txt 나누기

In [14]:
txt_files = listdir('./VCTK-Corpus-0.92/화자별/txt/all_txt')
print(len(txt_files))
txt_files[:5]

43885


['p225_001.txt',
 'p225_002.txt',
 'p225_003.txt',
 'p225_004.txt',
 'p225_005.txt']

In [15]:
data_path = './VCTK-Corpus-0.92/화자별/txt/'

In [16]:
for i in txt_files:
    split_name = i.split('.')[0]
    speaker = split_name[:4]
    for j in speaker_li_edit:
        if speaker == j:
            des_path_2 = data_path + '/' + j
            if not os.path.exists(des_path_2):
                os.makedirs(des_path_2)
            os.rename(data_path +'all_txt/'+i, des_path_2+'/'+i)

In [17]:
speakers = listdir('./VCTK-Corpus-0.92/화자별/txt/')
print(len(speakers))

109


In [34]:
for i in tqdm(speakers, mininterval=1):
    speaker_path = './VCTK-Corpus-0.92/화자별/txt/'+i
    file_li = listdir(speaker_path)
    txt_li = []
    for j in file_li:
        f = open(speaker_path+'/'+j, 'r')
        while True:
            line = f.readline()
            if not line: break
            txt_li.append(line.strip('\n'))
        f.close()
        df = pd.DataFrame(txt_li)
        df.to_excel("./VCTK-Corpus-0.92/화자별/excel/"+i+"_text.xlsx", encoding='utf-8-sig', header=None, index=False)


100%|██████████████████████████████████████████████████████████████████████████████| 109/109 [1:23:56<00:00, 46.20s/it]


In [2]:
import openpyxl as xl
import json
from collections import OrderedDict

In [3]:
data_path = './VCTK-Corpus-0.92/화자별/'

In [4]:
excel_files = listdir(data_path+'excel/')

In [5]:
print(len(excel_files))
print(excel_files[:5])

109
['p225_text.xlsx', 'p226_text.xlsx', 'p227_text.xlsx', 'p228_text.xlsx', 'p229_text.xlsx']


In [6]:
speakers = listdir(data_path+'wav/')

In [7]:
#def clean_text(text):
    #pattern = '(\.wav)$'
    #text = re.sub(pattern=pattern, repl='', string=text)
    #text = text.strip()
    #text = " ".join(text.split())
    #return text  

In [8]:
print(len(speakers))
print(speakers[:5])

109
['p225', 'p226', 'p227', 'p228', 'p229']


In [9]:
def clean_text(text):
    pattern = '(\.wav)$'
    text = re.sub(pattern=pattern, repl='', string=text)
    text = text.strip()
    text = " ".join(text.split())
    return text   

In [10]:
for i in  tqdm(excel_files, mininterval=1):
    split_name = i.split('_')[0]
    json_data = {}
    for j in speakers:
        if split_name == j:
            wb = xl.load_workbook(data_path+'excel/'+i)
            ws = wb.active
            wav_files = listdir(data_path+'wav/'+j)
            wav_li =[]
            for p in wav_files:
                q = "VCTK/"+j+"/item/"+p
                k = clean_text(q)
                wav_li.append(k)
            json_data = OrderedDict()
            for r in ws.rows:
                if str(r[0].value).isalpha():
                    continue
                if r[0].value is not None:
                    json_data[wav_li[r[0].value-1]] = r[1].value
            with open('vctk-'+j+'_alignment.json', 'w', encoding="utf-8") as make_file:
                json.dump(json_data, make_file, ensure_ascii=False, indent="\t")

100%|████████████████████████████████████████████████████████████████████████████████| 109/109 [00:07<00:00, 15.57it/s]
